# Pruebas de una sola grabación

In [1]:
import threading
import time
import csv
import numpy as np
import sounddevice as sd
import serial
import pygame
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import matplotlib.pyplot as plt
from ordered_set import OrderedSet

pygame 2.5.2 (SDL 2.28.3, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
import sys
sys.path.append('..')
# Custom imports
from analysis_utils import DataContainer
from audio_constants import ref_volume, ambient_volumes

In [5]:
ESP_COM_PORT = "COM23"
ESP_BAUDRATE = 115200

In [3]:
# Get the default audio playback device
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)

# Create a volume control interface
volume_controller = cast(interface, POINTER(IAudioEndpointVolume))

In [ ]:
# Sync variable between the serial and audio threads
current_volume = 0.0

In [15]:
# Function to receive serial data and store it
def receive_serial_data(data, volume_dependent):
    ser = serial.Serial(ESP_COM_PORT, ESP_BAUDRATE)  # Replace 'COM1' with your serial port
    while not terminate_event.is_set():
        received_data = ser.readline().decode().strip()
        received_data = received_data.split(" ")[0]
        try:
            received_data = float(received_data)
        except ValueError:
            received_data = float(received_data.split(".")[0])
        try:
            if log_flag:
                if not volume_dependent:
                    data = np.append(data, received_data)
                else:
                    data[current_volume] = np.append(data[current_volume], received_data)
        except KeyError:
            pass
    ser.close()

In [ ]:
# Function to play audio sine waves
def play_time_weighting_wave():
    duration = 10 # s
    sample_rate = 88200
    t = np.arange(0, duration, 1 / sample_rate)

    sine_wave = np.sin(2 * np.pi * 1000 * t)
    volume_controller.SetMasterVolumeLevelScalar(ref_db, None)

    log_flag = True

    # Wait before playing the sine wave
    time.sleep(0.375 if TIME_PERIOD == 0.125 else 3) #s

    sd.play(sine_wave, sample_rate, blocking=True)

    # Wait after playback is done
    time.sleep(20) #s

    log_flag = False

    # Send a signal to the other thread to terminate
    terminate_event.set()

# Function to play audio sine waves
def play_stability_wave():
    duration = 60 # s
    sample_rate = 88200
    t = np.arange(0, duration, 1 / sample_rate)

    sine_wave = np.sin(2 * np.pi * 1000 * t)
    volume_controller.SetMasterVolumeLevelScalar(ref_db, None)

    log_flag = True

    # Wait before playing the sine wave
    time.sleep(0.375 if TIME_PERIOD == 0.125 else 3) #s

    sd.play(sine_wave, sample_rate, blocking=True)

    # Wait after playback is done
    time.sleep(0.375 if TIME_PERIOD == 0.125 else 3) #s

    log_flag = False

    # Send a signal to the other thread to terminate
    terminate_event.set()

In [9]:
# Function to play ambient noise
def play_ambient_noise():
    # Initialize the pygame mixer
    pygame.mixer.init()

    # Load your MP3 file
    mp3_file = "people-talking-in-small-room-30s.mp3" if TIME_PERIOD == 0.125 else "people-talking-in-small-room-60s.mp3" 

    # Create a pygame Sound object from the MP3 file
    sound = pygame.mixer.Sound(mp3_file)

    for volume in ambient_volumes:
        global current_volume
        current_volume = volume
        volume_controller.SetMasterVolumeLevelScalar(volume, None)

        log_flag = True

        # Wait before playing the audio
        time.sleep(0.375 if TIME_PERIOD == 0.125 else 3) #s

        # Play the MP3 file
        sound.play()

        # Wait for the sound to finish
        pygame.time.delay(int(sound.get_length() * 1000))

        # Wait after playback is done
        time.sleep(0.375 if TIME_PERIOD == 0.125 else 3 ) #s

        log_flag = False

    # Send a signal to the other thread to terminate
    terminate_event.set()

    # Quit pygame mixer when done
    pygame.mixer.quit()

In [7]:
# Create formatted containers to store received serial data
time_weighting_data_F = np.array([])
time_weighting_data_S = np.array([])
stability_data = np.array([])

for volume in ambient_volumes:
    ambient_data_F[volume] = np.array([])
    ambient_data_S[volume] = np.array([])

# Event to signal termination
terminate_event = threading.Event()
# Flag to signal when to log measurements
log_flag = False

## Ejecutar pruebas

### Ponderación de tiempo con periodo F

In [ ]:
TIME_PERIOD = 0.125

# Create the serial data receiving thread
serial_thread = threading.Thread(target=receive_serial_data, args=(time_weighting_data_F,False), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_time_weighting_wave, daemon=True)

# Start serial thread
serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
serial_thread.join()
audio_thread.join()

# Reset the event for next test
terminate_event.clear()

### Ponderación de tiempo con periodo S

In [ ]:
TIME_PERIOD = 1.0

# Create the serial data receiving thread
serial_thread = threading.Thread(target=receive_serial_data, args=(time_weighting_data_S,False), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_time_weighting_wave, daemon=True)

# Start serial thread
serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
serial_thread.join()
audio_thread.join()

# Reset the event for next test
terminate_event.clear()

### Estabilidad

In [ ]:
TIME_PERIOD = 0.125

# Create the serial data receiving thread
serial_thread = threading.Thread(target=receive_serial_data, args=(stability_data, False), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_stability_wave, daemon=True)

# Start serial thread
serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
serial_thread.join()
audio_thread.join()

# Reset the event for next test
terminate_event.clear()

### Ruido ambiental con periodo F

In [ ]:
TIME_PERIOD = 0.125

# Create the serial data receiving thread
serial_thread = threading.Thread(target=receive_serial_data, args=(ambient_data_F, True), daemon = True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_ambient_noise, daemon = True)

# Start serial thread
serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
serial_thread.join()
audio_thread.join()

# Reset the event for next test
terminate_event.clear()

### Ruido ambiental con periodo S

In [ ]:
TIME_PERIOD = 1.0

# Create the serial data receiving thread
serial_thread = threading.Thread(target=receive_serial_data, args=(ambient_data_S, True), daemon=True)

# Create the audio playback thread
audio_thread = threading.Thread(target=play_ambient_noise, daemon=True)

# Start serial thread
serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
serial_thread.join()
audio_thread.join()

# Reset the event for next test
terminate_event.clear()

## Guardar datos

In [ ]:
with open("../results/single_shot_tests/t_wting_f.pkl", 'wb') as file:
    pickle.dump(time_weighting_data_F, file)

with open("../results/single_shot_tests/t_wting_s.pkl", 'wb') as file:
    pickle.dump(time_weighting_data_S, file)

with open("../results/single_shot_tests/stability.pkl", 'wb') as file:
    pickle.dump(stability_data, file)

with open("../results/single_shot_tests/ambient_f.pkl", 'wb') as file:
    pickle.dump(ambient_data_F, file)

with open("../results/single_shot_tests/ambient_s.pkl", 'wb') as file:
    pickle.dump(ambient_data_S, file)